In [ ]:
# Read in some data from the RDSF
import pathlib

device = "cuda"
my_ct_path = pathlib.Path(
    "~/zebrafish_rdsf/DATABASE/uCT/Wahab_clean_dataset/TIFS/ak_102.tif"
).expanduser()

assert my_ct_path.exists()

In [ ]:
import tifffile

scan = tifffile.imread(my_ct_path)

In [ ]:
from fishjaw.visualisation.images_3d import plot_slices

plot_slices(scan)

In [ ]:
# Crop out the jaw
import torch
from fishjaw.localisation.model import get_model, crop

model = get_model(device)
with open(
    "../../script_output/jaw_location/test68_with_shrinkage/test68_with_shrinkage.pth",
    "rb",
) as f:
    weights = torch.load(f)

model.load_state_dict(weights)
_ = model.eval()

In [ ]:
cropped = crop(model, scan, model_input_size=(512, 128, 128), window_size=(192, 192, 192))

In [ ]:
plot_slices(cropped)

In [ ]:
# Segment out the jaw
import torchio as tio

from fishjaw.model.model import load_model, predict
from fishjaw.model import data

# This was the median one when we trained lots of them, I think
inference_model = load_model("attempt_n3.pkl").load_model(set_eval=True)

# this should be abstracted
# It should also probably threshold, and take the largest connected component
# and live in the fishjaw.inference library
scaled = data.ints2float(cropped)
tensor = torch.as_tensor(scaled, dtype=torch.float32).unsqueeze(0)
subject = tio.Subject(image=tio.Image(tensor=tensor, type=tio.INTENSITY))
prediction = predict(
    inference_model,
    subject,
    patch_size=(160, 160, 160),
    patch_overlap=(4, 4, 4),
    activation="sigmoid",
) > 0.5

In [ ]:
_ = plot_slices(cropped, prediction)

In [ ]:
# Show a mesh of the jaw

In [ ]:
# Show the greyscale values of the jaw pixels

In [ ]:
# Show a histogram of the greyscale values in the jaw